# Convolutional Neural Networks
In this tutorial we will see how to define, train and visualize a Convolutional Neural Network (CNN) for image classification using the [Keras](https://keras.io/) library.

## From theory...

Recall (see [here](http://cs231n.github.io/convolutional-networks/) for details):

> Convolutional Neural Networks are very similar to ordinary Neural Networks from the previous chapter: they are made up of neurons that have learnable weights and biases. Each neuron receives some inputs, performs a dot product and optionally follows it with a non-linearity. So what does change? ConvNet architectures make the explicit assumption that the inputs are images, which allows us to encode certain properties into the architecture. These then make the forward function more efficient to implement and vastly reduce the amount of parameters in the network.

Most CNNs have a feed-forward architecture and are tipically composed of [Convolutional](https://keras.io/layers/convolutional/), [Pooling](https://keras.io/layers/pooling/) and [Fully-Connected](https://keras.io/layers/core/#dense) layers.

CNN models first demonstrated their effectiveness for task of classification. In this context, a [Softmax](https://keras.io/activations/) activation after the last Dense layer encodes the probability distribution over the classes and the loss function employed is the categorical [cross-entropy](https://keras.io/objectives/).

More recently CNNs have been successfully employed to tackle a huge variety of problems, including e.g. semantic segmentation, visual attention, edge detection, image and video captioning etc. (see e.g. [here](https://github.com/kjw0612/awesome-deep-vision) for some examples). In order to deal with these different tasks, network architectures have evolved too from simple feed-forward to arbitrarily complex models.

## ...to practice.

[Keras](https://keras.io/) library provides facility for both the [feed-forward](https://keras.io/models/sequential/) and for [more complex](https://keras.io/getting-started/functional-api-guide/) architectures. The latter can be used for defining composite models, such as multi-output models, directed acyclic graphs, or models with shared layers.

**In this tutorial, we train a simple feed-forward CNN to perform image classification** on the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset, which consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. For this task, a [sequential](https://keras.io/models/sequential/) model is good enough: however, we'll tincker with the [functional API](https://keras.io/getting-started/functional-api-guide/) in order to acquire confidence with both syntaxes. Let's start!

In [ ]:
from __future__ import print_function
from keras.datasets import cifar10
from keras.utils import np_utils

# cifar 10 classes
nb_classes = 10

# input image dimensions
img_rows, img_cols, img_channels = 32, 32, 3

# The data, shuffled and split between train and test sets:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalize X in range (0, 1)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

print('X_train shape:', X_train.shape)
print('Train samples: {}    Test samples{}.'.format(X_train.shape[0], X_test.shape[0])

# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

### Defining the model (sequential API)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

### Defining the model (functional API)

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten 
from keras.layers import Convolution2D, MaxPooling2D

model_in = Input(shape=X_train.shape[1:])

x = Convolution2D(32, 3, 3, activation='relu', border_mode='same') (model_in)
x = Convolution2D(32, 3, 3,  activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(x)
x = Convolution2D(64, 3, 3, activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(nb_classes)(x)
model_out = Activation('softmax')(x)

model = Model(input=model_in, output=model_out)

### Compiling the model

Keras provides a variety of different [optimizers](https://keras.io/optimizers/).

In [ ]:
from keras.optimizers import SGD

# Let's train the model using SGD + momentum:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

### Let the training begin!

Now we are ready to actually start the training of the network. First we set some parameters, in particular the **batch size** is the size of the batch used to perform the gradient descent. We can also decide to leverage on Keras [support](https://keras.io/preprocessing/image/) for **data augmentation**, which is always performed in CPU in a separate thread. 

In [ ]:
# training parameters
batch_size = 128
nb_epoch = 200
data_augmentation = True

 Now we can really start. If everything has gone well the loss should gets down to 0.65 (test logloss) in 25 epochs, and down to 0.55 after 50 epochs.

In [1]:
from keras.preprocessing.image import ImageDataGenerator

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))

Using TensorFlow backend.


NameError: name 'data_augmentation' is not defined